In [ ]:
import pwd
pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
      root_dir: Path
      data_path: Path
      model_ckpt: str
      num_train_epochs: int
      warmup_steps: int
      per_device_train_batch_size: int
      weight_decay: float
      logging_steps: int
      eval_strategy: str
      eval_steps: int
      save_steps: int
      gradient_accumulation_steps: int

In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_trainer
    
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            eval_strategy=params.eval_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        
        return model_trainer_config



In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [ ]:
#This is the Normal model to train the model


# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config

#     def train(self):
#         # 🔸 Force CPU to avoid MPS OOM
#         device = "cpu"

#         tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
#         model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

#         seq2seq_data_collator = DataCollatorForSeq2Seq(
#             tokenizer,
#             model=model_pegasus
#         )

#         # Load dataset
#         dataset_samsum_pt = load_from_disk(self.config.data_path)

#         training_args = TrainingArguments(
#             output_dir=self.config.root_dir,
#             num_train_epochs=self.config.num_train_epochs,
#             warmup_steps=self.config.warmup_steps,
#             per_device_train_batch_size=self.config.per_device_train_batch_size,
#             per_device_eval_batch_size=self.config.per_device_train_batch_size,
#             weight_decay=self.config.weight_decay,
#             logging_steps=self.config.logging_steps,
#             eval_strategy=self.config.eval_strategy,
#             eval_steps=self.config.eval_steps,
#             save_steps=self.config.save_steps,
#             gradient_accumulation_steps=self.config.gradient_accumulation_steps,
#             no_cuda=True,            # 🔸 don’t use CUDA
#             use_mps_device=False     # 🔸 don’t use MPS
#         )

#         trainer = Trainer(
#             model=model_pegasus,
#             args=training_args,
#             tokenizer=tokenizer,
#             data_collator=seq2seq_data_collator,
#             train_dataset=dataset_samsum_pt["train"],       # 🔸 use train, not test
#             eval_dataset=dataset_samsum_pt["validation"]
#         )

#         trainer.train()

#         # Save model
#         model_pegasus.save_pretrained(ocs.path.join(self.config.root_dir, "pegasus-samsum-model"))

#         # Save tokenizer
#         tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        print("\n⚡ FAST TEST MODE ENABLED — Using t5-small + tiny dataset\n")

        device = "cpu"
        SMALL_MODEL = "t5-small"

        tokenizer = AutoTokenizer.from_pretrained(SMALL_MODEL)
        model = AutoModelForSeq2SeqLM.from_pretrained(SMALL_MODEL).to(device)

        def preprocess_fn(batch):
            inputs = tokenizer(
                batch["dialogue"],
                max_length=256,        # 🔸 must fit T5
                truncation=True
            )
            labels = tokenizer(
                batch["summary"],
                max_length=64,
                truncation=True
            )
            inputs["labels"] = labels["input_ids"]
            return inputs

        dataset = load_from_disk(self.config.data_path)

        # 🔸 Take 10 samples
        tiny_train = dataset["train"].select(range(10)).map(preprocess_fn)
        tiny_val   = dataset["validation"].select(range(10)).map(preprocess_fn)

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            logging_steps=1,
            eval_strategy="steps",
            eval_steps=2,
            save_steps=10000,
            warmup_steps=0,
            weight_decay=0.0,
            gradient_accumulation_steps=1,

            # CPU only
            use_cpu=True,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            tokenizer=tokenizer,
            train_dataset=tiny_train,
            eval_dataset=tiny_val
        )

        trainer.train()

        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e